**I JUST SHOULD RUN THE ```PIP``` INSTALLATION TO TRAIN MY MODEL** 

In [3]:
from transformers import AutoModelForSeq2SeqLM,AutoModelForSequenceClassification, AutoTokenizer
import re
from rdkit.Chem import MolFromSmiles
import string
from rdkit import RDLogger 
import pandas as pd
from transformers import T5ForSequenceClassification, T5Config
import torch

import numpy as np


/home/tetkin/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


*Мое говно*

In [4]:
train_data = pd.DataFrame(pd.read_csv('../train_split_fluor.csv'))[['Chromophore','Solvent','Absorption max (nm)','Emission max (nm)','Quantum yield']].dropna()
validate_data  = pd.DataFrame(pd.read_csv('../test_split_fluor.csv'))[['Chromophore','Solvent','Absorption max (nm)','Emission max (nm)','Quantum yield']].dropna()

df = pd.DataFrame(train_data).dropna()
df["input_text"] = df["Chromophore"] + " " + df["Solvent"]

# Separating features and targets
X = df["input_text"].tolist()
#, "Emission max (nm)", "Quantum yield"
y = df[["Absorption max (nm)"]].values.tolist()

# Random shuffle the data
np.random.seed(42)  # For reproducibility
indices = np.arange(len(X))
np.random.shuffle(indices)

# Split the indices for training and evaluation
split = int(len(indices) * 0.8)  # 60% train, 20% eval
train_indices = indices[:split]
eval_indices = indices[split:]

# Create train and eval datasets
X_train = [X[i] for i in train_indices]
y_train = [y[i] for i in train_indices]  # Use train_indices directly
X_eval = [X[i] for i in eval_indices]
y_eval = [y[i] for i in eval_indices]  # Use eval_indices directly

train_labels = torch.tensor(y_train).float()
eval_labels = torch.tensor(y_eval).float()

**Не мое говно**

In [5]:
# from datasets import load_dataset
# ds = load_dataset("csv", data_files="../train_split_fluor.csv")

# def preprocess(d):
#     prompt = add_special_symbols(f"{d['Solvent']}, {d['Chromophore']}")
#     inputs = tokenizer(prompt,padding="longest", max_length=512, truncation=True, return_tensors="pt")
#     inputs["input_ids"] = inputs["input_ids"][0]
#     inputs["attention_mask"] = inputs["attention_mask"][0]
#     inputs["label"] = d["Absorption max (nm)"]
#     return inputs

# train = ds["train"].map(preprocess, batched=False)

In [6]:
RDLogger.DisableLog('rdApp.*')
atoms_tokens = ['Ag','Al','As','Au','B','Ba','Bi','Br','C','Ca',
              'Cd','Cl','Co','Cr','Cs','Cu','F','Fe','Ga','Gd',
              'Ge','H','Hg','I','In','K','Li','M','Mg','Mn',
              'Mo','N','Na','O','P','Pt','Ru','S','Sb','Sc',
              'Se','Si','Sn','V','W','Z','Zn','c','e','n','o','p','s']
atoms_tokens = sorted(atoms_tokens, key=lambda s: len(s), reverse=True)
SMI_REGEX_PATTERN = r"(\[|\]|\(|\)|\.|=|#|-|\+|\\|\/|:|~|@|\?|>>?|\*|\$|\%[0-9]{2}|[0-9]|" + \
                                                                  '|'.join(atoms_tokens) + ")"
regex = re.compile(SMI_REGEX_PATTERN)
def clean_output_sequence(output_sequence):
    return output_sequence.replace('</s>', '').replace('<sm_', '').replace(' sm_', '').replace('>', '').strip()
def add_special_symbols(text):
  output = []
  for word in text.split():
      tokens = [token for token in regex.findall(word)]
      if len(tokens) > 4 and (word == ''.join(tokens)) and MolFromSmiles(word):
          output.append(''.join(['<sm_'+t+'>' for t in tokens]))
      else:
          output.append(word)
  return ' '.join(output)
PROMPT = "Given the following reactants and reagents, please provide a possible product. " + (str)(X_train[0])
          
          
PROMPT = add_special_symbols(PROMPT)

config = T5Config.from_pretrained('insilicomedicine/nach0_base')
config.num_labels=1
model = T5ForSequenceClassification.from_pretrained('insilicomedicine/nach0_base',
 config=config,
 ignore_mismatched_sizes=True)
tokenizer = AutoTokenizer.from_pretrained('insilicomedicine/nach0_base')



Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at insilicomedicine/nach0_base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)
        
train_dataset = CustomDataset(tokenizer(X_train), train_labels)
eval_dataset = CustomDataset(tokenizer(X_eval), eval_labels)

In [8]:
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    
# Training arguments
training_args = TrainingArguments(
    output_dir="./output",
    # run_name=run_name,
    learning_rate=1,
    per_device_train_batch_size=4,
    # per_device_train_batch_size=32,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="tensorboard",
    resume_from_checkpoint=False,
    lr_scheduler_type="cosine",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset =eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
)
trainer.train()

/home/tetkin/.venv/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 12.00 MiB. GPU 3 has a total capacity of 31.73 GiB of which 1.44 MiB is free. Process 114147 has 446.00 MiB memory in use. Process 124414 has 16.48 GiB memory in use. Process 387586 has 13.57 GiB memory in use. Process 409845 has 396.00 MiB memory in use. Including non-PyTorch memory, this process has 872.00 MiB memory in use. Of the allocated memory 491.76 MiB is allocated by PyTorch, and 6.24 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

OutOfMemoryError: CUDA out of memory. Tried to allocate 12.00 MiB. GPU 3 has a total capacity of 31.73 GiB of which 1.44 MiB is free. Process 114147 has 446.00 MiB memory in use. Process 124414 has 16.48 GiB memory in use. Process 387586 has 13.57 GiB memory in use. Process 409845 has 396.00 MiB memory in use. Including non-PyTorch memory, this process has 872.00 MiB memory in use. Of the allocated memory 491.77 MiB is allocated by PyTorch, and 6.23 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)